### Octree Visualization Implementation

In [114]:
import pandas as pd
import random

In [124]:
# House
# data = pd.read_csv('house_random.csv', sep=',')[['X', 'Y', 'Z']]
# data = pd.read_csv('house_spatial.csv', sep=',')[['X', 'Y', 'Z']]

# Tree
# data = pd.read_csv('tree_random.csv', sep=',')[['X', 'Y', 'Z']]
# data = pd.read_csv('tree_spatial.csv', sep=',')[['X', 'Y', 'Z']]

# Light Pole
# data = pd.read_csv('lightpole_random.csv', sep=',')[['X', 'Y', 'Z']]
data = pd.read_csv('data/LightPole/csv/spatial/lightpole_spatial.csv', sep=',')[['X', 'Y', 'Z']]

FileNotFoundError: [Errno 2] No such file or directory: 'lightpole_spatial.csv'

In [116]:
len(data)

4096

In [117]:
# This is a function for creating unique id for each node

node_ids = set()

def generate_node_id(node_ids):
    new_id = random.randint(1, 1e5)
    while new_id in node_ids:
        new_id = random.randint(1, 1e5)
    node_ids.add(new_id)
    return new_id

In [118]:
# Stores node information
octree = []


# Main function
def visualize_octree(data, source_id, depth):
    
    # Base case
    if depth == 3 or len(data) == 0:
        return
    
    global node_ids
    global octree
    
    # Calculate the boundaries
    min_x, max_x = data['X'].min(), data['X'].max()
    min_y, max_y = data['Y'].min(), data['Y'].max()
    min_z, max_z = data['Z'].min(), data['Z'].max()

    # Calculate midpoints
    mid_x = (max_x + min_x) / 2
    mid_y = (max_y + min_y) / 2
    mid_z = (max_z + min_z) / 2

    # The eight octants in 3D space
    subdivisions = [
        (data['X'] <= mid_x) & (data['Y'] <= mid_y) & (data['Z'] <= mid_z),
        (data['X'] <= mid_x) & (data['Y'] <= mid_y) & (data['Z'] > mid_z),
        (data['X'] <= mid_x) & (data['Y'] > mid_y) & (data['Z'] <= mid_z),
        (data['X'] <= mid_x) & (data['Y'] > mid_y) & (data['Z'] > mid_z),
        (data['X'] > mid_x) & (data['Y'] <= mid_y) & (data['Z'] <= mid_z),
        (data['X'] > mid_x) & (data['Y'] <= mid_y) & (data['Z'] > mid_z),
        (data['X'] > mid_x) & (data['Y'] > mid_y) & (data['Z'] <= mid_z),
        (data['X'] > mid_x) & (data['Y'] > mid_y) & (data['Z'] > mid_z),
    ]
    
    # We are reaching further depth
    depth = depth + 1

    for i, subdivision in enumerate(subdivisions):
        filtered_data = data[subdivision]
        num_points_in_box = len(filtered_data)
        
        target_id = generate_node_id(node_ids)
        
        # Create box to hold each piece of data
        box = [num_points_in_box, depth, source_id, target_id]
        
        # Store box info
        if (num_points_in_box > 0):
            octree.append(box)
        
        # Further division
        visualize_octree(filtered_data, target_id, depth)

In [119]:
visualize_octree(data, source_id=0, depth=0)

In [120]:
octree_df = pd.DataFrame(octree, columns=['Degree', 'Depth', 'Source', 'Target'])

In [121]:
# Edge
# Source,Target,Type

octree_df_edge = octree_df[['Source', 'Target']].copy()
octree_df_edge['Type'] = 'Directed'

In [122]:
# Node
# Id,Degree(num_point),Depth

octree_df_node = octree_df[['Target', 'Degree', 'Depth']]
octree_df_node = octree_df_node.rename(columns={'Target': 'Id'})
octree_df_node.loc[len(octree_df_node)] = {'Id': 0, 'Degree': 4096, 'Depth': 0} # append root node

In [123]:
# export CSV

octree_df_edge.to_csv('data/LightPole/csv/spatial/edge.csv', index = False)
octree_df_node.to_csv('data/LightPole/csv/spatial/node.csv', index = False)